In [30]:
from Packages import RegistrationFunc
import torch
import matplotlib.pyplot as plt
from skimage import data

In [31]:
t = torch.from_numpy(data.astronaut()[:,:,0])

original

In [32]:
def get_div(v):
    v_x = (torch.roll(v[0], shifts=(0, -1), dims=(0, 1)) - torch.roll(v[0], shifts=(0, 1), dims=(0, 1)))/2
    v_y = (torch.roll(v[1], shifts=(-1, 0), dims=(0, 1)) - torch.roll(v[1], shifts=(1, 0), dims=(0, 1)))/2
    return v_x + v_y


def get_det(jac_m): # size_h x size_w
    return jac_m[0, 0] * jac_m[1, 1] - jac_m[0, 1] * jac_m[1, 0] #


def get_jacobian_determinant(diffeo): # diffeo: 2 x size_h x size_w
    jac_m = get_jacobian_matrix(diffeo) # jac_m: 2 x 2 x size_h x size_w
    return jac_m[0, 0] * jac_m[1, 1] - jac_m[0, 1] * jac_m[1, 0] # size_h x size_w


def get_jacobian_matrix(diffeo): # diffeo: 2 x size_h x size_w
#     return torch.stack((get_gradient(diffeo[1]), get_gradient(diffeo[0])))
    print(get_gradient(diffeo[0]).shape)
    return torch.stack((get_gradient(diffeo[0]), get_gradient(diffeo[1])))


def get_gradient(F):  # 2D F: size_h x size_w
    F_padded = torch.zeros((F.shape[0]+2,F.shape[1]+2))
    F_padded[1:-1,1:-1] = F
    F_padded[0,:] = F_padded[1,:]
    F_padded[-1,:] = F_padded[-2,:]
    F_padded[:,0] = F_padded[:,1]
    F_padded[:,-1] = F_padded[:,-2]
    F_x = (torch.roll(F_padded, shifts=(0, -1), dims=(0, 1)) - torch.roll(F_padded, shifts=(0, 1), dims=(0, 1)))/2
    F_y = (torch.roll(F_padded, shifts=(-1, 0), dims=(0, 1)) - torch.roll(F_padded, shifts=(1, 0), dims=(0, 1)))/2
    return torch.stack((F_x[1:-1,1:-1].type(torch.DoubleTensor), F_y[1:-1,1:-1].type(torch.DoubleTensor)))

In [33]:
a = get_div(get_gradient(t))

In [34]:
aj = get_jacobian_matrix(get_gradient(t))

torch.Size([2, 512, 512])


In [35]:
aj.shape

torch.Size([2, 2, 512, 512])

In [36]:
ag = get_gradient(t)

modified

In [37]:
def get_div(v):
    v_x = (torch.roll(v[0], shifts=(-1, 0), dims=(0, 1)) - torch.roll(v[0], shifts=(1, 0), dims=(0, 1)))/2
    v_y = (torch.roll(v[1], shifts=(0, -1), dims=(0, 1)) - torch.roll(v[1], shifts=(0, 1), dims=(0, 1)))/2
    return v_x + v_y


def get_det(jac_m): # size_h x size_w
    return jac_m[0, 0] * jac_m[1, 1] - jac_m[0, 1] * jac_m[1, 0] #


def get_jacobian_determinant(diffeo): # diffeo: 2 x size_h x size_w
    jac_m = get_jacobian_matrix(diffeo) # jac_m: 2 x 2 x size_h x size_w
    return jac_m[0, 0] * jac_m[1, 1] - jac_m[0, 1] * jac_m[1, 0] # size_h x size_w


def get_jacobian_matrix(diffeo): # diffeo: 2 x size_h x size_w
#     return torch.stack((get_gradient(diffeo[1]), get_gradient(diffeo[0])))
    return torch.stack((get_gradient(diffeo[0]), get_gradient(diffeo[1])))


def get_gradient(F):  # 2D F: size_h x size_w
    F_padded = torch.zeros((F.shape[0]+2,F.shape[1]+2))
    F_padded[1:-1,1:-1] = F
    F_padded[0,:] = F_padded[1,:]
    F_padded[-1,:] = F_padded[-2,:]
    F_padded[:,0] = F_padded[:,1]
    F_padded[:,-1] = F_padded[:,-2]
    F_x = (torch.roll(F_padded, shifts=(-1, 0), dims=(0, 1)) - torch.roll(F_padded, shifts=(1, 0), dims=(0, 1)))/2
    F_y = (torch.roll(F_padded, shifts=(0, -1), dims=(0, 1)) - torch.roll(F_padded, shifts=(0, 1), dims=(0, 1)))/2
    return torch.stack((F_x[1:-1,1:-1].type(torch.DoubleTensor), F_y[1:-1,1:-1].type(torch.DoubleTensor)))

In [38]:
b = get_div(get_gradient(t))

In [39]:
bj = get_jacobian_matrix(get_gradient(t))

In [40]:
bg = get_gradient(t)

In [46]:
%matplotlib widget
torch.norm(aj[0,1]-bj[1,0])

tensor(0., dtype=torch.float64)

In [2]:
a=torch.ones((3,100,120))
torch.stack((a,a,a)).shape

torch.Size([3, 3, 100, 120])

In [51]:
HH, WW, DD = torch.meshgrid([torch.arange(11),#, dtype=torch.double
                                 torch.arange(12),#, dtype=torch.double
                                 torch.arange(13)])#, dtype=torch.double
# original and 4.3
c=torch.stack((HH, WW, DD)).double() 

In [52]:
c.shape

torch.Size([3, 11, 12, 13])

In [55]:
c[:,10,11,12]

tensor([10., 11., 12.], dtype=torch.float64)